<a href="https://colab.research.google.com/github/ianuhs/FinSentiNet/blob/main/FinSentiNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Financial Sentiment Analysis using NLP

Tech Stack:


Python
* Programming language of choice

PyTorch / CUDA
* Deep learning framework and tools

DistilBERT
* distilbert-base-uncased (6 layers, uncased, 66M parameters)

Hugging Face
* Pre-trained models, datasets, and training APIs

Dataset
* Financial PhraseBank
* Split dataset: train/validation/test (60/20/20)

Transformers
* Implements pre-trained NLP models (DistilBERT)
* Handles tokenization, batching, and model I/O

Scikit-learn
* Evaluation metrics: accuracy, precision, recall, FQ
* Label performance metrics

NumPy
* Computations, arrays, and reproducible randomness

Matplotlib
* Visualization of metrics and training results



# Set Up


In [ ]:
# Upgrades
!pip install --upgrade transformers datasets --quiet

# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score, roc_auc_score
from sklearn.preprocessing import label_binarize
from datasets import Value, ClassLabel

# Warning suppression
import warnings
import logging
warnings.filterwarnings("ignore")
logging.getLogger("transformers").setLevel(logging.ERROR)

# Hugging Face Transformers & Datasets
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding)
from datasets import Dataset, DatasetDict

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Retrieve data
file_paths = [
    '/content/Sentences_50Agree.txt',
    '/content/Sentences_66Agree.txt',
    '/content/Sentences_75Agree.txt',
    '/content/Sentences_AllAgree.txt'
]

texts = []
labels = []

for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                # Split last occurrence of '@' to separate text and label
                text, label = line.rsplit('@', 1)
                texts.append(text)
                labels.append(label)
            except ValueError:
                print(f"Skipping malformed line in {file_path}: {line}")

# Convert to pandas DataFrame
df = pd.DataFrame({"text": texts, "label": labels})

# Map sentiment labels to integers
label_names = ["negative", "neutral", "positive"]
label_to_id = {label: i for i, label in enumerate(label_names)}
df["label_id"] = df["label"].map(label_to_id)
if df["label_id"].isnull().any():
    raise ValueError("Some labels in your dataset do not match label_names!")
df["label_id"] = df["label_id"].astype(int)

# Split into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df["label_id"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label_id"])

# Convert to Hugging Face DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False),
    "test": Dataset.from_pandas(test_df, preserve_index=False)
})

print(dataset)

# Tokenizer & data collator
MODEL_CHECKPOINT = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_function(batch):
    tokenized = tokenizer(batch["text"], truncation=True, padding=True)
    tokenized["labels"] = batch["label_id"]
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text", "label", "label_id"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Baseline Model

* 5 min for 2 epochs



In [ ]:
# Load DistilBERT
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels = len(label_names)  # label_names = ["negative", "neutral", "positive"]
).to(device)  # Move model to GPU

# Define metric function for Hugging Face Trainer
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Training arguments for Hugging Face Trainer
training_args = TrainingArguments(
    output_dir="./phase1_baseline",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32, # For T4 GPU memory
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

# Initialize Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

# Evaluate on validation set
eval_results = trainer.evaluate()
print("\n=== Baseline Model Validation Metrics ===")
for k, v in eval_results.items():
    if k.startswith("eval_"):
        print(f"{k}: {v:.4f}")

# Performance Metrics and Visuals

log_history = trainer.state.log_history

train_epochs, train_loss, train_acc = [], [], []
eval_epochs, eval_loss, eval_acc = [], [], []

for log in log_history:
    if 'loss' in log:
        train_epochs.append(log.get('epoch'))
        train_loss.append(log.get('loss'))
        train_acc.append(log.get('accuracy', np.nan))  # may be NaN if not logged
    if 'eval_loss' in log:
        eval_epochs.append(log.get('epoch'))
        eval_loss.append(log.get('eval_loss'))
        eval_acc.append(log.get('eval_accuracy'))

# === Plot training and evaluation loss ===
plt.figure(figsize=(10,6))
plt.plot(train_epochs, train_loss, label="Train Loss", color="tab:red")
plt.plot(eval_epochs, eval_loss, label="Eval Loss", color="tab:orange")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Evaluation Loss per Epoch")
plt.legend()
plt.show()

# === Plot training and evaluation accuracy ===
plt.figure(figsize=(10,6))
plt.plot(train_epochs, train_acc, label="Train Accuracy", color="tab:blue")
plt.plot(eval_epochs, eval_acc, label="Eval Accuracy", color="tab:green")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.title("Training and Evaluation Accuracy per Epoch")
plt.legend()
plt.show()

# === Predictions for test set ===
preds_output = trainer.predict(tokenized_dataset["test"])
preds = np.argmax(preds_output.predictions, axis=-1)
labels = preds_output.label_ids

# === ROC-AUC (multi-class, one-vs-rest) ===
labels_binarized = label_binarize(labels, classes=list(range(len(label_names))))
preds_proba = torch.nn.functional.softmax(torch.tensor(preds_output.predictions), dim=1).numpy()
roc_auc = roc_auc_score(labels_binarized, preds_proba, average="weighted", multi_class="ovr")
print(f"\nBaseline Model Weighted ROC-AUC: {roc_auc:.4f}")

# === Confusion Matrix ===
cm = confusion_matrix(labels, preds)
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(cm, cmap="Blues")
ax.set_xticks(range(len(label_names)))
ax.set_yticks(range(len(label_names)))
ax.set_xticklabels(label_names)
ax.set_yticklabels(label_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Baseline Model Confusion Matrix")

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha="center", va="center", color="red", fontsize=14)
plt.show()

# TO-DO

Performance metrics from ground up. What am I currently recording? How often should record? What should actually be recorded? Show per epoch but track epoch/2?